# EQA results scraper

UKNEQAS provide external quality assessment (EQA) services for UK clinical laboratories. 

Laboratories submit results that are compared against target values, and a PDF report is provided in a standard format.

Sometimes, a laboratory may wish to use the results and target values for other purposes (for example, if the EQA samples are analysed as part of a validation of a new method). However, it is not easy to quickly get this information from a PDF file.

The following functions extract information from UKNEQAS EQA reports.

In [1]:
import PyPDF2
import pandas as pd
import tabula

## Extract distribution summary from EQA report

Opens a UKNEQAS report, and extracts the results and target values from the Distribution Summary pages as a Pandas dataframe.

In [2]:
def eqa_dist_sum(file):
    '''
    Get the distribution summary from a UKNEQAS EQA report
    '''
    #Open and read the PDF file, and create an emptry Pandas dataframe
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    df = pd.DataFrame()

    #Open each page in the PDF document and extract text
    for page in range(1, pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        text = pageObj.extractText()
        
    #If the page contains the distribution summary...
        if 'Distribution Summary' in text:
            # ... read the distribution summary table
            dist_sum = tabula.read_pdf(file, stream=True, pages=(page+1)
                        ,area=(19,0,75,40), relative_area=True
                        )
            # ... read and add the scheme name
            scheme_read = tabula.read_pdf(file, stream=True, pages=(page+1)
                    ,area=(0,28,3,77), relative_area=True
                    ,pandas_options={'header': None})
            scheme_name = scheme_read[0].iloc[0,0]
            
            if scheme_name[0:4] == "for ":
                   scheme_name = scheme_name[4:]
            
            dist_sum[0]['Scheme name'] = scheme_name
            
            #... and add to the dataframe
            df = pd.concat([df,dist_sum[0]],ignore_index=True)

    # Rename the first column
    
    df = df.rename(columns={"Unnamed: 0": "Analyte"})
    
    # Counts the number of samples in the distribution, and copies the analyte name to empty rows
    no_specimens = df['Specimen'].nunique()
    for specimen_no in range(1,no_specimens):
        df['Analyte'][specimen_no::no_specimens] = df['Analyte'][0::no_specimens]
    
    return df

In [3]:
eqa_dist_sum("NEQAS TFTS 436 20190205.pdf")

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:42]


,Analyte,Specimen,Pool,Result,Targ,Scheme name
0,TSH,436A,983,1.78,1.8,Thyroid Hormones
1,TSH,436B,892,7.06,6.4,Thyroid Hormones
2,TSH,436C,893,8.29,7.7,Thyroid Hormones
3,TSH,436D,894,10.59,9.8,Thyroid Hormones
4,TSH,436E,959,0.46,0.5,Thyroid Hormones
5,Free T4,436A,983,14.30,14.4,Thyroid Hormones
6,Free T4,436B,892,11.20,11.1,Thyroid Hormones
7,Free T4,436C,893,7.60,8.2,Thyroid Hormones
8,Free T4,436D,894,4.50,5.2,Thyroid Hormones
9,Free T4,436E,959,14.30,15.1,Thyroid Hormones


In [4]:
eqa_dist_sum("4000 UKNEQAS Newborn British Isles 314.pdf")

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:42]


,Analyte,Specimen,Pool,Result,Targ,Scheme name
0,Initial Phe,314A,659,475,445,Birmingham Quality ~ Newborn British Isles
1,Initial Phe,314B,660,637,537,Birmingham Quality ~ Newborn British Isles
2,Initial Phe,314C,661,652,640,Birmingham Quality ~ Newborn British Isles
3,Initial Phe interpretation,314A,659,F,F,Birmingham Quality ~ Newborn British Isles
4,Initial Phe interpretation,314B,660,F,F,Birmingham Quality ~ Newborn British Isles
5,Initial Phe interpretation,314C,661,F,F,Birmingham Quality ~ Newborn British Isles
6,Final Phe,314A,659,491,440,Birmingham Quality ~ Newborn British Isles
7,Final Phe,314B,660,618,528,Birmingham Quality ~ Newborn British Isles
8,Final Phe,314C,661,670,626,Birmingham Quality ~ Newborn British Isles
9,Tyr,314A,659,147,129,Birmingham Quality ~ Newborn British Isles


## Multi report extractor

If we have many EQA reports to extract, then we can place them all in a subfolder, and extract results from all reports in that folder.

The outputs are combined in a single Pandas dataframe.

In [5]:
import os

In [6]:
folder_location = '.\EQA reports'
if not os.path.exists(folder_location):os.mkdir(folder_location)

In [9]:
def eqa_dist_sum_multi():
    combined = pd.DataFrame()

    for file in os.listdir(folder_location):
        individual_results = eqa_dist_sum(os.path.join(folder_location,file))
        combined = pd.concat([combined,individual_results],ignore_index=True)
        print("Results extracted from " + str(file))
    
    print("All files complete")
    return combined

In [10]:
combined = eqa_dist_sum_multi()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:42]


Results extracted from 4500 UKNEQAS Newborn British Isles 321.pdf
Results extracted from UKNEQAS Immunosuppressants 137.pdf
Results extracted from UKNEQAS Urine Catecholamines 246.pdf
All files complete


The dataframe can be exported to a csv file using .to_csv

In [11]:
combined.to_csv('EQA_results_and_targets.csv', index=False)

## Analyte detail extractor

The analyte detail section of the EQA report contains additional detail, such as the standard uncertainty of the target value.

EQAAnalyteSum() attempts to capture this additional information from each quantitative analyte page of the report, and returns a Pandas dataframe.

This is a little trickier, as the information included in this section can vary according to the scheme.

The distribution number is included in the output, but the specimen number (e.g. A, B, C etc.) is not.

Since a separate page must be read for each analyte, this function takes significantly longer to run than reading the distribution summary.


In [12]:
def eqa_analyte_sum(file):
    
    #Open and read the PDF file, and create an emptry Pandas dataframe
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    df = pd.DataFrame()
    print("Opening " + file)
    #Open each page in the PDF document and extract text
    for page in range(1, pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        text = pageObj.extractText()
        print("...Opening page " + str(page))
        
    #If the page contains an analyte page (but isn't an "interpretation")...
        if ('Analyte : ') in text and ('Pool description / Treatments / Additions' in text) and ('interpretation' not in text):
            print('  --- Analyte page found')
            # ... read the analyte summary table
            analyte_sum = tabula.read_pdf(file, stream=True, pages=(page+1)
                    ,area=(17,76,75,97), relative_area=True
                     ,pandas_options={'header': None, 'columns':('Name','Value')})
            
            # ... read and add the analyte name
            analyte_read = tabula.read_pdf(file, stream=True, pages=(page+1)
                    ,area=(6,28,8,77), relative_area=True
                    ,pandas_options={'header': None})
            analyte_name = analyte_read[0].iloc[0,0][10:]
            analyte_sum[0]['Analyte name'] = analyte_name
            
            # ... read and add the distribution
            dist_read = tabula.read_pdf(file, stream=True, pages=(page+1)
                    ,area=(3.5,28,5,47), relative_area=True
                    ,pandas_options={'header': None})
            dist = dist_read[0].iloc[0,0][15:]
            analyte_sum[0]['Distribution'] = dist
                                          
            # ... read and add the scheme name
            scheme_read = tabula.read_pdf(file, stream=True, pages=(page+1)
                    ,area=(0,28,3,77), relative_area=True
                    ,pandas_options={'header': None})
            scheme_name = scheme_read[0].iloc[0,0]
            
            if scheme_name[0:4] == "for ":
                   scheme_name = scheme_name[4:]
            
            analyte_sum[0]['Scheme name'] = scheme_name
            
            #... and add to the dataframe
            df = pd.concat([df,analyte_sum[0]],ignore_index=True)
            
    print(file + " scrape complete")    
    return df
    

In [13]:
eqa_analyte_sum("4000 UKNEQAS Newborn British Isles 314.pdf")

Opening 4000 UKNEQAS Newborn British Isles 314.pdf
...Opening page 1
...Opening page 2
...Opening page 3
...Opening page 4
...Opening page 5
...Opening page 6
...Opening page 7
...Opening page 8
  --- Analyte page found
...Opening page 9
...Opening page 10
...Opening page 11
  --- Analyte page found
...Opening page 12
...Opening page 13
  --- Analyte page found
...Opening page 14
...Opening page 15
...Opening page 16
  --- Analyte page found
...Opening page 17
...Opening page 18
...Opening page 19
  --- Analyte page found
...Opening page 20
...Opening page 21
  --- Analyte page found
...Opening page 22
...Opening page 23
  --- Analyte page found
...Opening page 24
...Opening page 25
...Opening page 26
  --- Analyte page found
...Opening page 27
...Opening page 28
  --- Analyte page found
...Opening page 29
...Opening page 30
  --- Analyte page found
...Opening page 31
...Opening page 32
  --- Analyte page found
...Opening page 33
4000 UKNEQAS Newborn British Isles 314.pdf scrape comple

,Name,Value,Analyte name,Distribution,Scheme name
0,Your result,475.0,Initial Phe (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
1,Target value,445.0,Initial Phe (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
2,(ALTM),NaN,Initial Phe (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
3,Standard Uncertainty,13.0,Initial Phe (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
4,Your specimen:,NaN,Initial Phe (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
...,...,...,...,...,...
325,%bias,-16.4,Met (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
326,transformed bias,NaN,Met (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
327,Accuracy Index,NaN,Met (umol/L WB),314,Birmingham Quality ~ Newborn British Isles
328,Notional 'Spike',30.0,Met (umol/L WB),314,Birmingham Quality ~ Newborn British Isles


## Analyte detail extractor - multiple files

If we have many EQA reports to extract, then we can place them all in a subfolder, and extract results from all reports in that folder.

The outputs are combined in a single Pandas dataframe.

**Warning - this may take a while.**

In [14]:
import os

In [15]:
folder_location = '.\EQA reports'
if not os.path.exists(folder_location):os.mkdir(folder_location)

In [16]:
def eqa_analyte_multi():
    combined = pd.DataFrame()

    for file in os.listdir(folder_location):
        individual_results = eqa_analyte_sum(os.path.join(folder_location,file))
        combined = pd.concat([combined,individual_results],ignore_index=True)
        print("Results extracted from " + str(file))
    
    print("All files complete")    
    return combined

In [17]:
analyte_multi = eqa_analyte_multi()

Opening .\EQA reports\4500 UKNEQAS Newborn British Isles 321.pdf
...Opening page 1
...Opening page 2
...Opening page 3
...Opening page 4
...Opening page 5
...Opening page 6
...Opening page 7
  --- Analyte page found
...Opening page 8
...Opening page 9
...Opening page 10
  --- Analyte page found
...Opening page 11
...Opening page 12
...Opening page 13
  --- Analyte page found
...Opening page 14
...Opening page 15
...Opening page 16
  --- Analyte page found
...Opening page 17
...Opening page 18
...Opening page 19
  --- Analyte page found
...Opening page 20
...Opening page 21
...Opening page 22
  --- Analyte page found
...Opening page 23
...Opening page 24
  --- Analyte page found
...Opening page 25
...Opening page 26
...Opening page 27
  --- Analyte page found
...Opening page 28
...Opening page 29
...Opening page 30
  --- Analyte page found
...Opening page 31
...Opening page 32
  --- Analyte page found
...Opening page 33
...Opening page 34
  --- Analyte page found
...Opening page 35
...O

The dataframe can be exported to a csv file using .to_csv

In [18]:
analyte_multi.to_csv('EQA_analyte_information.csv', index=False)